In [ ]:
!nvidia-smi

Fri Jun 17 22:22:27 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install transformers
!pip install simpletransformers

In [ ]:
!gdown --id 1lSu7e7BmF1xUHIVMldWuKBh_10Qbsgi_ #multi_modal_classification_model
!mv "/content/multi_modal_classification_model.py" "/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/multi_modal_classification_model.py"

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1lSu7e7BmF1xUHIVMldWuKBh_10Qbsgi_
To: /content/multi_modal_classification_model.py
100% 59.4k/59.4k [00:00<00:00, 62.6MB/s]


NOTE: Restart runtime after running previous cells

#Setup and Config

In [ ]:
!gdown --id 1ZGIHE4DF1Nb316jPBWyU7Zzp_K1Lp5xM

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1ZGIHE4DF1Nb316jPBWyU7Zzp_K1Lp5xM
To: /content/TaskA-Memes.zip
100% 188M/188M [00:03<00:00, 57.6MB/s]


In [ ]:
!unzip "/content/TaskA-Memes.zip"
!rm TaskA-Memes.zip

In [ ]:
import sklearn
import pandas as pd
import numpy as np
import torch
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from simpletransformers.classification import MultiModalClassificationModel, MultiModalClassificationArgs
import seaborn as sns

In [ ]:
import logging

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [ ]:
seed = 41

np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

#Data

In [ ]:
df = pd.read_csv('/content/TaskA-Memes/task_a.csv')

In [ ]:
df.drop(columns=['Unnamed: 0'], inplace=True)

In [ ]:
df = df[df.meme_type != 'only offensive']

In [ ]:
df.reset_index(inplace=True, drop=True)

In [ ]:
df['hate_speech'] = [1 if meme == 'hate_speech' else 0 for meme in df.meme_type]
df.drop(columns='meme_type', inplace=True)

In [ ]:
df

,file_name,text,hate_speech
0,img24.jpg,انا نظري ضعيف ومش هخش الجيش اقسم بالله انت قناص,1
1,img28.jpg,لما فيمينست تتابع هانتر وتعرف غون بيبحث ابوه و...,1
2,img10.jpg,لا أحد النسويات,1
3,img7.jpg,يعني لو قلعنا الحجاب وتعرينا هنطلع القمر أوماااال,1
4,img16.jpg,فيمينيست عربية نفسي اطلع اوروبا حرية وتحرر و ح...,1
...,...,...,...
1669,IMG_269.jpg,لاتوجد نساء اجمل منك مهما بحثت والله؟! يعني بح...,0
1670,IMG_233.jpg,أنا أخوض حرب لا أعلم عنها شيئا,0
1671,IMG_138.jpg,لما اروح عند دكتور الاسنان يادكتور بالله عليك ...,0
1672,IMG_705.jpg,الضربة اللي متموتش تقوي لأ انا عايزاللي تموت,0


In [ ]:
train_images = df.file_name
train_text = df.text
train_labels = df.hate_speech

In [ ]:
train_data = []

for i in range(len(df)):
  train_data.append([train_images[i], train_text[i], train_labels[i]])

In [ ]:
train_data = pd.DataFrame(train_data, columns=['images','text','labels'])
train_data

,images,text,labels
0,img24.jpg,انا نظري ضعيف ومش هخش الجيش اقسم بالله انت قناص,1
1,img28.jpg,لما فيمينست تتابع هانتر وتعرف غون بيبحث ابوه و...,1
2,img10.jpg,لا أحد النسويات,1
3,img7.jpg,يعني لو قلعنا الحجاب وتعرينا هنطلع القمر أوماااال,1
4,img16.jpg,فيمينيست عربية نفسي اطلع اوروبا حرية وتحرر و ح...,1
...,...,...,...
1669,IMG_269.jpg,لاتوجد نساء اجمل منك مهما بحثت والله؟! يعني بح...,0
1670,IMG_233.jpg,أنا أخوض حرب لا أعلم عنها شيئا,0
1671,IMG_138.jpg,لما اروح عند دكتور الاسنان يادكتور بالله عليك ...,0
1672,IMG_705.jpg,الضربة اللي متموتش تقوي لأ انا عايزاللي تموت,0


In [ ]:
train_df, test_df = train_test_split(train_data, test_size=0.1, random_state=41)

In [ ]:
test_df

,images,text,labels
708,img1044.jpg,الرب يخلق اليهود الرب ايضا يخلق عيسى اليهود يق...,1
31,img22.jpg,لما الضيوف يقعدوا ساكتين لربك نبعد مولين جماعة...,0
472,img501.jpg,شبكة نتفليكس عمل جيد لكن لماذا هذا الفيلم لا ي...,1
1360,IMG_920.jpg,2019 :شخص كسول لا فائدة منه 2020:مواطن مالح و ...,0
660,img1013.png,المسلم بي لايك النظرة الاولى حلال,1
...,...,...,...
1562,IMG_1012.jpg,الحلاقين حاليا,0
1405,IMG_498.jpg,لما ارتبط ب واحد مش بتاع بنات وبيصلي بيحبني ان...,0
206,img245.png,حقوق أساسية للمرأة الزلام هل هذه سيابة,1
522,img544.jpg,فرنسا تنهب ثروات الشعوب الافريقية وتتسبب تفقير...,1


In [ ]:
print(len(train_df))
print(len(test_df))

1506
168


# Model

In [ ]:
model_args = MultiModalClassificationArgs(
    num_train_epochs=3,
    fp16=False,
    train_batch_size=32,
    learning_rate=3e-5,
    max_seq_length=32,
    save_best_model=True,
    overwrite_output_dir=True,
    save_model_every_epoch=False,
    gradient_accumulation_steps=1,
)

In [ ]:
model = MultiModalClassificationModel(
  'bert',
  'bert-base-multilingual-uncased',
  use_cuda=True,
  label_list=[0,1],
  args=model_args
) 

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
model.config.use_return_dict = False

In [ ]:
model.train_model(
        train_df, 
        image_path='/content/TaskA-Memes/'
)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 0 of 3:   0%|          | 0/48 [00:00<?, ?it/s]

Running Epoch 1 of 3:   0%|          | 0/48 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/48 [00:00<?, ?it/s]

INFO:simpletransformers.classification.multi_modal_classification_model: Training of bert model complete. Saved to outputs/.


(144, 0.43427009771888453)

In [ ]:
import sklearn

result, model_outputs = model.eval_model(
    test_df,
    acc=sklearn.metrics.accuracy_score,
    image_path='/content/TaskA-Memes/',
)

Running Evaluation:   0%|          | 0/21 [00:00<?, ?it/s]

INFO:simpletransformers.classification.multi_modal_classification_model:{'mcc': 0.6641574223058726, 'tp': 66, 'tn': 73, 'fp': 21, 'fn': 8, 'acc': 0.8273809523809523, 'eval_loss': 0.4227278803785642}


In [ ]:
test_df

,images,text,labels
708,img1044.jpg,الرب يخلق اليهود الرب ايضا يخلق عيسى اليهود يق...,1
31,img22.jpg,لما الضيوف يقعدوا ساكتين لربك نبعد مولين جماعة...,0
472,img501.jpg,شبكة نتفليكس عمل جيد لكن لماذا هذا الفيلم لا ي...,1
1360,IMG_920.jpg,2019 :شخص كسول لا فائدة منه 2020:مواطن مالح و ...,0
660,img1013.png,المسلم بي لايك النظرة الاولى حلال,1
...,...,...,...
1562,IMG_1012.jpg,الحلاقين حاليا,0
1405,IMG_498.jpg,لما ارتبط ب واحد مش بتاع بنات وبيصلي بيحبني ان...,0
206,img245.png,حقوق أساسية للمرأة الزلام هل هذه سيابة,1
522,img544.jpg,فرنسا تنهب ثروات الشعوب الافريقية وتتسبب تفقير...,1


In [ ]:
# model_outputs

In [ ]:
_, preds = torch.max(torch.tensor(model_outputs), dim=1)

preds

tensor([1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1,
        1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1,
        1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0,
        0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0,
        1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0,
        0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1,
        1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1])

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score

labels = np.array(test_df.labels)

print("F1 macro:   {}".format(round(f1_score(labels, preds, average="macro"), 3)))
print("Precision:   {}".format(precision_score(labels,preds, average="macro"), 3))
print("Recall:   {}".format(recall_score(labels,preds, average="macro"), 3))
print("F1 micro:   {}".format(round(f1_score(labels, preds, average="micro"), 3)))